<a href="https://colab.research.google.com/github/tmdoi/small-Japanese-LLM-compare/blob/main/benchMarkOrginal_v08math.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install "transformers>=4.43" accelerate torch --upgrade
!pip -q install pandas sacrebleu rouge-score fugashi ipadic

In [ ]:
!pip install -q google-genai

In [ ]:
import os
from google.colab import userdata

# SDK 読み込み
from google import genai

# Colab Secrets から安全に取得
api_key = userdata.get('GEMINI_API_KEY')

if not api_key:
    raise ValueError("GEMINI_API_KEY が定義されていません。Colab の Secrets を確認してください。")


client = genai.Client(api_key=api_key)

In [ ]:
# === (Colab セル1) セットアップ & モデル読込 ===
try:
    import google.colab  # noqa: F401
    IN_COLAB = True
except Exception:
    IN_COLAB = False

import sys, subprocess, math, time, re
def pip_install(pkgs):
    cmd = [sys.executable, "-m", "pip", "install", "-q", "--upgrade"] + pkgs
    print("Installing:", " ".join(pkgs))
    subprocess.check_call(cmd)

# 必要に応じて有効化してください（初回実行時など）
# pip_install(["transformers>=4.43", "accelerate", "torch", "pandas", "sacrebleu", "rouge-score", "fugashi", "ipadic"])

# ---- 以降 Python 本体 ----
import torch, pandas as pd
from dataclasses import dataclass
from typing import List, Dict, Any, Tuple
from transformers import AutoModelForCausalLM, AutoTokenizer
from rouge_score import rouge_scorer
import sacrebleu
import numpy as np
import random

# 乱数固定（再現性の一助）
random.seed(0)
np.random.seed(0)
if torch.cuda.is_available():
    torch.manual_seed(0)
    torch.cuda.manual_seed_all(0)

# 比較対象モデル（必要に応じて変更可）
MODELS = {
    "RakutenAI-2.0-mini-instruct": "Rakuten/RakutenAI-2.0-mini-instruct",
    "TinySwallow-1.5B-Instruct":   "SakanaAI/TinySwallow-1.5B-Instruct",
}

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

@dataclass
class GenConfig:
    max_new_tokens: int = 256
    temperature: float = 0.0   # 再現性重視
    top_p: float = 1.0
    do_sample: bool = False
    num_beams: int = 1

GENCFG = GenConfig()

def load_model(repo_id: str) -> Tuple[AutoTokenizer, AutoModelForCausalLM]:
    tok = AutoTokenizer.from_pretrained(repo_id, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(
        repo_id,
        torch_dtype="auto",
        device_map="auto",   # ColabのGPUに自動割当
    )
    return tok, model

def chat_generate(tokenizer, model, messages: List[Dict[str, str]], cfg: GenConfig = GENCFG):
    # 各モデルのchatテンプレートを利用
    input_ids = tokenizer.apply_chat_template(
        messages, add_generation_prompt=True, return_tensors="pt"
    ).to(model.device)

    attn = None
    if tokenizer.pad_token_id is not None:
        attn = input_ids.ne(tokenizer.pad_token_id).long()

    t0 = time.perf_counter()
    with torch.no_grad():
        out_ids = model.generate(
            input_ids,
            max_new_tokens=cfg.max_new_tokens,
            do_sample=cfg.do_sample,
            temperature=cfg.temperature,
            top_p=cfg.top_p,
            num_beams=cfg.num_beams,
            attention_mask=attn,
            pad_token_id=tokenizer.eos_token_id,
        )
    dt = time.perf_counter() - t0
    gen_ids = out_ids[:, input_ids.shape[1]:]
    text = tokenizer.batch_decode(gen_ids, skip_special_tokens=True)[0].strip()
    toks = gen_ids.shape[1]
    tps = toks / dt if dt > 0 else float("nan")
    return text, {"latency_sec": dt, "gen_tokens": toks, "tok_per_sec": tps}

# タスク定義（自動採点可能なもの中心）
TASKS = [
    {
        "name": "JA-QA: 富士山の標高",
        "messages": [
            {"role":"system","content":"あなたは有能な日本語アシスタントです。"},
            {"role":"user","content":"富士山の標高は？数値と単位で簡潔に答えてください。"}
        ],
        # 3776 を数値として含めれば正解扱い（ゆるい判定）
        "judge": lambda x: ("3776" in re.sub(r"[^\d]", "", x)) or ("3,776" in x) or ("3776 m" in x) or ("3776メートル" in x),
    },
    {
        "name": "算数: 12×(7+5)",
        "messages": [
            {"role":"system","content":"あなたは計算に正確です。"},
            {"role":"user","content":"12×(7+5) の結果だけを半角数字で答えてください。"}
        ],
        "judge": lambda x: "144" in re.sub(r"[^\d\-]", "", x),
    },
    {
        "name": "要約: 5文→1文",
        "messages": [
            {"role":"system","content":"与えられた段落を1文で要約してください。"},
            {"role":"user","content":
             "奈良公園には多くのシカが生息し、観光客に人気です。"
             "近年は観光客の増加に伴い、エサの与え方やごみ問題が課題となっています。"
             "地元自治体はルール啓発と清掃活動を強化しています。"
             "一方で来園者のマナー向上には時間がかかるとの指摘もあります。"
             "持続可能な観光の実現に向け、地域と来訪者の協力が求められています。"
            }
        ],
        "ref": "奈良公園のシカと観光をめぐる課題に対し、自治体と来訪者の協力による持続可能な観光の実現が求められている。",
        "rougeL": True
    },
    {
        "name": "翻訳: EN→JA",
        "messages": [
            {"role":"system","content":"次の英文を自然な日本語に翻訳してください。"},
            {"role":"user","content":"Edge-friendly small LLMs enable private, low-latency applications without relying on cloud services."}
        ],
        "ref": "エッジ向けの小型LLMは、クラウドサービスに依存せずにプライバシーに配慮した低遅延アプリケーションを可能にする。",
        "bleu": True
    },
]

# ROUGE-L スコアラー（※ベンチマーク側でも参照）
scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=False)

# モデル／トークナイザを事前ロードして保持（ベンチマーク側で再利用）
print("\n== Loading models ==")
LOADED = {}  # { model_name: (tokenizer, model) }
for name, repo in MODELS.items():
    print(f"Loading: {name} ({repo})")
    tok, mdl = load_model(repo)
    LOADED[name] = (tok, mdl)

print("Loaded:", list(LOADED.keys()))


In [ ]:
TASKS=[
    {
        "name": "要約: 教育",
        "messages": [
            {"role":"system","content":"与えられた段落を1文で要約してください。"},
            {"role":"user","content":
             "教育は社会の基盤を支える重要な要素であり、"
             "知識や技術の習得だけでなく、人間性の成長にも寄与する。"
             "現代社会ではICT活用が進み、新しい学びの形が模索されている。"
            }
        ],
        "ref": "教育は知識・技術と人間性の成長を支える基盤であり、ICT活用による新しい学びが求められている。",
        "rougeL": True
    },
    {
        "name": "翻訳: JA→EN",
        "messages": [
            {"role":"system","content":"次の日本語を自然な英語に翻訳してください。"},
            {"role":"user","content":"大阪は日本で三番目に大きな都市です。"}
        ],
        "ref": "Osaka is the third largest city in Japan.",
        "bleu": True
    },
    ]

In [ ]:
# === (Colab セルTaskAdd) 四則演算タスク30題を既存TASKSに追加 ===
import re

def _eq_num(ans: str, expect: str) -> bool:
    # 半角数字・符号のみを抽出して比較（負数対応）
    norm = re.sub(r"[^\d\-]", "", str(ans)).lstrip("+")
    return norm == expect

EXTRA_TASKS = [
    { "name": "算数: 12+35",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"12+35 の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "47")
    },
    { "name": "算数: 58-19",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"58-19 の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "39")
    },
    { "name": "算数: 13×7",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"13×7 の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "91")
    },
    { "name": "算数: 96÷12",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"96÷12 の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "8")
    },
    { "name": "算数: 23+48-15",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"23+48-15 の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "56")
    },
    { "name": "算数: 12×(7+5)",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"12×(7+5) の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "144")
    },
    { "name": "算数: (45-17)÷7",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"(45-17)÷7 の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "4")
    },
    { "name": "算数: 9×9+27",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"9×9+27 の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "108")
    },
    { "name": "算数: 120÷(5×3)",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"120÷(5×3) の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "8")
    },
    { "name": "算数: 3×(14-8)+10",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"3×(14-8)+10 の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "28")
    },
    { "name": "算数: 200-4×45",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"200-4×45 の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "20")
    },
    { "name": "算数: (64÷8)+7×5",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"(64÷8)+7×5 の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "43")
    },
    { "name": "算数: (15+25+35)÷5",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"(15+25+35)÷5 の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "15")
    },
    { "name": "算数: 11×11-50",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"11×11-50 の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "71")
    },
    { "name": "算数: 18÷3+27÷9",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"18÷3+27÷9 の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "9")
    },
    { "name": "算数: (100-28)÷9",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"(100-28)÷9 の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "8")
    },
    { "name": "算数: 7×(8+6)-20",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"7×(8+6)-20 の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "78")
    },
    { "name": "算数: 40÷(6+2)",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"40÷(6+2) の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "5")
    },
    { "name": "算数: 3×(5×4)",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"3×(5×4) の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "60")
    },
    { "name": "算数: (81÷9)×(14-11)",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"(81÷9)×(14-11) の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "27")
    },
    { "name": "算数: 2×2×2×5",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"2×2×2×5 の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "40")
    },
    { "name": "算数: 90÷(15÷3)",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"90÷(15÷3) の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "18")
    },
    { "name": "算数: 7+(48÷6)×3",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"7+(48÷6)×3 の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "31")
    },
    { "name": "算数: (72-36)÷(9-3)",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"(72-36)÷(9-3) の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "6")
    },
    { "name": "算数: 50-(3×4+2)",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"50-(3×4+2) の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "36")
    },
    { "name": "算数: (18+24+30)÷6",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"(18+24+30)÷6 の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "12")
    },
    { "name": "算数: 14×(3+2×4)",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"14×(3+2×4) の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "154")
    },
    { "name": "算数: (100÷4)÷5",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"(100÷4)÷5 の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "5")
    },
    { "name": "算数: 8×(9-7×3)",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"8×(9-7×3) の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "-96")
    },
    { "name": "算数: (5-9)×(16÷4)",
      "messages": [
          {"role":"system","content":"あなたは計算に正確です。"},
          {"role":"user","content":"(5-9)×(16÷4) の結果だけを半角数字で答えてください。"}
      ],
      "judge": lambda x: _eq_num(x, "-16")
    },
]

# 既存 TASKS に追加
TASKS.extend(EXTRA_TASKS)
print(f"追加完了: TASKS は {len(TASKS)} 件になりました（今回追加: {len(EXTRA_TASKS)} 件）。")


In [ ]:
# --- 依存関係 ---
# pip install google-genai==0.3.* rapidfuzz

import os, re, json, time, hashlib, math
from typing import List, Dict, Any, Optional

# ==== 0) Gemini 初期化（AI Studio キーを環境変数で） ====
from google import genai
from google.genai.types import Tool, GenerateContentConfig

GEMINI_MODEL = "gemini-2.5-pro"  # 使うモデル。必要に応じて変更
import os
from google import genai

# シークレットから取得
api_key = "GEMINI_API_KEY"
if not api_key:
    raise ValueError("GEMINI_API_KEY が Colab のシークレットに設定されていません。")

# クライアント初期化
client = genai.Client(api_key=api_key)
print("✅ Gemini API キーをシークレットから読み込みました。")

# ==== 1) キャッシュ ====
CACHE_PATH = "extract_cache.json"
try:
    with open(CACHE_PATH, "r") as f:
        EXTRACT_CACHE: Dict[str, Any] = json.load(f)
except:
    EXTRACT_CACHE = {}

def _key(s: str) -> str:
    return hashlib.sha256(s.encode("utf-8")).hexdigest()

def _cache_get(ans: str) -> Optional[Any]:
    return EXTRACT_CACHE.get(_key(ans))

def _cache_set(ans: str, value: Any):
    EXTRACT_CACHE[_key(ans)] = value

def save_cache():
    with open(CACHE_PATH, "w") as f:
        json.dump(EXTRACT_CACHE, f, ensure_ascii=False, indent=2)

# ==== 2) まずは正規表現でローカル抽出 ====
# - 負号、小数、分数( a/b )、百分率( % )、全角→半角 の簡易対応
# - 複数候補があれば最後のものを採用（要件に応じてルール変更）
zen2han = str.maketrans("０１２３４５６７８９－．％／", "0123456789-./%")

FRACTION = re.compile(r"(-?\d+)\s*/\s*(\d+)")
NUMBER   = re.compile(r"([-+]?\d+(?:\.\d+)?)")
PERCENT  = re.compile(r"([-+]?\d+(?:\.\d+)?)\s*%")

def local_numeric_extract(text: str) -> Optional[str]:
    s = text.translate(zen2han)

    # 明示された「答え」「最終」付近を優先（ヒューリスティック）
    focus = None
    for anchor in ["答え", "最終", "結果", "よって", "したがって"]:
        i = s.rfind(anchor)
        if i != -1:
            focus = s[i:i+80]
            break
    target = focus or s

    # 分数（a/b）優先
    m = FRACTION.findall(target)
    if m:
        num, den = m[-1]
        # 必要に応じて既約化/小数化可。ここは「表記を保つ」ため文字列で返す。
        return f"{num}/{den}"

    # 百分率
    m = PERCENT.findall(target)
    if m:
        return m[-1]  # 例: "12.5"（%は落とす。必要であれば別タグで保持）

    # 通常の数値
    m = NUMBER.findall(target)
    if m:
        return m[-1]

    # だめなら全文から拾う
    m = FRACTION.findall(s) or PERCENT.findall(s) or NUMBER.findall(s)
    if m:
        if isinstance(m[0], tuple):
            num, den = m[-1]
            return f"{num}/{den}"
        return m[-1] if isinstance(m[-1], str) else None

    return None

# ==== 3) Gemini でバッチ抽出（JSON で厳密に返させる） ====
# 期待フォーマット: {"results": [{"value": "<string or number or NA>"} ...]}
def gemini_extract_many(texts: List[str], max_retries=4, rpm=2) -> List[Optional[str]]:
    # レート制限（例: 1分あたり2リクエスト）
    # ここでは「1バッチ=1リクエスト」として守る。呼び出し側でバッチ化すること。
    # 429時は指数バックオフ。
    # JSONスキーマを軽く強制するため、出力形式を固定。

    # プロンプト（厳格指示）
    sys_prompt = (
        "あなたは日本語の数式回答から最終数値のみを抽出して返す抽出器です。"
        "以下の各回答について、次のルールで結果を JSON 配列で返してください。\n"
        "ルール:\n"
        "1) 最終的な数値解のみ。単位や余分な文字は除外。\n"
        "2) 分数は 'a/b' の文字列で返す（既約化は不要）。\n"
        "3) 数値が一意に定まらなければ 'NA' を返す。\n"
        "4) 出力は JSON: {\"results\": [{\"value\": ...}, ...]} の形。\n"
    )

    # 入力の構築
    items = [{"id": i, "text": t} for i, t in enumerate(texts)]
    user_prompt = "対象テキスト一覧:\n" + "\n".join(
        [f"[{it['id']}] {it['text']}" for it in items]
    ) + "\n\n" + "以上です。順番を変えずに results を返してください。"

    # 応答を JSON に限定
    cfg = GenerateContentConfig(
        response_mime_type="application/json",
    )

    for attempt in range(max_retries):
        try:
            resp = client.models.generate_content(
                model=GEMINI_MODEL,
                contents=[{"role":"user","parts":[{"text": sys_prompt + "\n" + user_prompt}]}],
                config=cfg,
            )
            data = resp.text
            parsed = json.loads(data)
            out = []
            for i in range(len(texts)):
                try:
                    v = parsed["results"][i]["value"]
                    # number の場合も文字列化して整合
                    out.append(str(v) if v is not None else None)
                except Exception:
                    out.append(None)
            return out
        except Exception as e:
            # 429 等
            msg = str(e)
            if "429" in msg or "RESOURCE_EXHAUSTED" in msg:
                wait = (2 ** attempt) * 15  # 15s, 30s, 60s, 120s...
                time.sleep(wait)
                continue
            else:
                # その他の失敗は即座に None 配列
                return [None] * len(texts)

    return [None] * len(texts)

# ==== 4) 統合：まずキャッシュ→ローカル→（必要時のみ）Gemini ====
def extract_numeric_answer(answer_text: str) -> Optional[str]:
    if not answer_text:
        return None
    cached = _cache_get(answer_text)
    if cached is not None:
        return cached

    # まずローカル抽出
    local = local_numeric_extract(answer_text)
    if local is not None:
        _cache_set(answer_text, local)
        return local

    # Gemini（単発呼び出し用に gemini_extract_many を流用）
    res = gemini_extract_many([answer_text])
    value = res[0] if res else None
    _cache_set(answer_text, value)
    save_cache()
    return value

# ==== 5) 評価ループ側での使い方（例） ====
# - 既存 evaluate_one_loaded() 内で、SLM生回答 -> extract_numeric_answer() に差し替え
# - まとめて抽出したい場合は、バッチ化で gemini_extract_many() を呼ぶ（件数×APIコール削減）

# 例: バッチ抽出のフロー（Pseudo）
def batch_extract(answers: List[str], batch_size: int = 8) -> List[Optional[str]]:
    results = []
    batch = []
    idxs  = []

    # 1) まずキャッシュ/ローカルで埋める
    for i, ans in enumerate(answers):
        v = _cache_get(ans) or local_numeric_extract(ans)
        if v is not None:
            _cache_set(ans, v)
            results.append(v)
        else:
            results.append(None)
            batch.append(ans)
            idxs.append(i)

    # 2) Gemini が必要な分だけまとめて投げる
    if batch:
        gem = gemini_extract_many(batch)
        for j, val in enumerate(gem):
            i = idxs[j]
            results[i] = val
            _cache_set(answers[i], val)
        save_cache()
    return results


In [ ]:
# === (Colab セル2) 日本語対応のベンチマーク実行（セル1は無改変） ===
import math, numpy as np, pandas as pd, unicodedata, re
import sacrebleu

# セル1で定義済みの以下を利用します:
# - LOADED: { model_name: (tokenizer, model) }
# - TASKS: タスクリスト
# - chat_generate(): 生成関数
# - GENCFG: 生成設定
# ※ セル1の `scorer` は使わず、セル2内で日本語対応スコアラーを用意します。

# ----------------------------
# 日本語テキストの前処理 & トークナイザ設定（セル2内完結）
# ----------------------------
def normalize_ja(s: str) -> str:
    # 全角半角の揺れ・不要空白などを吸収（必要に応じて調整）
    s = unicodedata.normalize("NFKC", s)
    s = s.strip()
    return s

# fugashi (MeCab) が使えれば単語ベース、無ければ文字ベース
try:
    from fugashi import Tagger
    _tagger = Tagger()
    def ja_tokens(text: str):
        text = normalize_ja(text)
        return [m.surface for m in _tagger(text)]
    _bleu_tokenize = "ja-mecab"  # sacrebleu の日本語用トークナイザ指定
    print("Tokenizer for ROUGE: fugashi(MeCab) / BLEU: ja-mecab")
except Exception:
    def ja_tokens(text: str):
        text = normalize_ja(text)
        return list(text)  # 文字ベース
    _bleu_tokenize = "char"       # 文字ベースBLEU
    print("Tokenizer for ROUGE: char-level / BLEU: char")

# ----------------------------
# ROUGE-L（日本語向け）の実装（セル2内完結）
# ----------------------------
# rouge_score の内部Tokenizerはセル1で固定されている可能性があるため、
# ここではLCSに基づくROUGE-L F1をセル2側で実装します。
def _lcs_len(a, b):
    # a, b: トークン列
    # 動的計画法でLCS長を計算（O(n*m)）
    n, m = len(a), len(b)
    dp = [0]*(m+1)
    for i in range(1, n+1):
        prev = 0
        for j in range(1, m+1):
            tmp = dp[j]
            if a[i-1] == b[j-1]:
                dp[j] = prev + 1
            else:
                dp[j] = max(dp[j], dp[j-1])
            prev = tmp
    return dp[m]

def rougeL_f1(ref: str, hyp: str) -> float:
    ref_toks = ja_tokens(ref)
    hyp_toks = ja_tokens(hyp)
    if len(ref_toks) == 0 or len(hyp_toks) == 0:
        return 0.0
    lcs = _lcs_len(ref_toks, hyp_toks)
    prec = lcs / len(hyp_toks)
    rec  = lcs / len(ref_toks)
    if prec + rec == 0:
        return 0.0
    f1 = (2 * prec * rec) / (prec + rec)
    return float(f1)

# ----------------------------
# 評価関数（セル1のモデル群を利用）
# ----------------------------
def evaluate_one_loaded(model_name: str, tok_mdl):
    tok, mdl = tok_mdl
    rows = []
    for task in TASKS:
        out, stats = chat_generate(tok, mdl, task["messages"])
        # 軽いノイズ除去（任意）：丁寧な前置きのカット例
        out_clean = re.sub(r"^(はい、|承知しました。|以下のとおりです。)+", "", out).strip()
        response = extract_numeric_answer(out_clean)
                    # client.models.generate_content( model="gemini-2.5-pro",
                    # contents="次の数式を含む文字列から答えのみ数値として取り出したい．数値だけを出力して．「"+ out_clean +"」" )
        out_clean = response
        print(out_clean)

        row = {
            "model": model_name,
            "task": task["name"],
            "output": out_clean+"<="+out,      # 生出力も保持
            **stats
        }
        # pass@1（セル1定義のjudgeに従う）
        if "judge" in task:
            row["pass@1"] = bool(task["judge"](out_clean))

        # ROUGE-L（セル2内で日本語対応計算）
        if task.get("rougeL"):
            r = rougeL_f1(task["ref"], out_clean)
            row["ROUGE-L"] = r

        # BLEU（日本語向けトークナイズ設定）
        if task.get("bleu"):
            bleu = sacrebleu.corpus_bleu(
                [normalize_ja(out_clean)],
                [[normalize_ja(task["ref"])]],
                tokenize=_bleu_tokenize
            ).score
            row["BLEU"] = bleu

        rows.append(row)
    return pd.DataFrame(rows)

# ----------------------------
# 実行
# ----------------------------
all_dfs = []
for name, tok_mdl in LOADED.items():
    print(f"\n== Evaluating {name} ==")
    df = evaluate_one_loaded(name, tok_mdl)
    display(df[["model","task","pass@1","ROUGE-L","BLEU","latency_sec","tok_per_sec","output"]])
    all_dfs.append(df)

summary = pd.concat(all_dfs, ignore_index=True)

# 集計（タスク別の平均）
def safe_mean(xs):
    xs = [x for x in xs if x is not None and not (isinstance(x, float) and math.isnan(x))]
    return float(np.mean(xs)) if xs else float("nan")

report = []
for m in summary["model"].unique():
    sub = summary[summary["model"]==m]
    pass_mean = safe_mean([1.0 if x is True else (0.0 if x is False else None) for x in sub.get("pass@1", []).tolist()])
    rouge_mean = safe_mean(sub.get("ROUGE-L", []).tolist())
    bleu_mean  = safe_mean(sub.get("BLEU", []).tolist())
    tps_mean   = safe_mean(sub.get("tok_per_sec", []).tolist())
    lat_mean   = safe_mean(sub.get("latency_sec", []).tolist())
    report.append({
        "model": m,
        "pass@1(mean)": pass_mean,
        "ROUGE-L(mean)": rouge_mean,
        "BLEU(mean)": bleu_mean,
        "tok_per_sec(mean)": tps_mean,
        "latency_sec(mean)": lat_mean
    })

print("\n== Summary ==")
display(pd.DataFrame(report))

# 生成条件を変更したい場合（任意）
# GENCFG.max_new_tokens = 128
# GENCFG.temperature = 0.7
# GENCFG.do_sample = True
# print("New GenConfig:", GENCFG)